Task 4a:
- Implement a Locality Sensitive Hashing (LSH) tool (for Euclidean distance) which takes as input (a) the number of layers, L, (b) the number of hashes per layer, h, and (c) a set of vectors as input and creates an in-memory index structure containing the given set of vectors. See
  - ”Near-Optimal Hashing Algorithms for Approximate Nearest Neighbor in High Dimensions” (by Alexandr Andoni and Piotr Indyk). Communications of the ACM vol. 51, no. 1, 2008, pp. 117-122.

In [2]:
import numpy as np
import random
import itertools
import math

import pandas as pd
import pickle

import json
from torchvision.datasets import Caltech101
from torchvision.models  import resnet50, ResNet50_Weights
from torchvision import models, transforms
from scipy.special import softmax

import matplotlib.pyplot as plt
from scipy.spatial.distance import euclidean, cosine, minkowski, correlation
from PIL import Image
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN

from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import collections

In [3]:
path = '/content/drive/MyDrive/CSE515_Phase3'

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
caltech_dataset = Caltech101(root = f'{path}/data', download = False)

In [7]:
json_file = f"{path}/feature_descriptors.json"
with open(json_file, 'r') as file:
  feature_descriptors = json.load(file)

data = pd.DataFrame(feature_descriptors).T.reset_index(names="id")

In [ ]:
class LSH:
  def __init__(self, L, h, input_vectors):
    self.L = L
    self.h = h
    self.W = 0.0
    self.vectors = input_vectors
    self.lsh_hyperplanes = {}
    self.lsh_hyperplane_ranges = {}
    self.hash_tables = [{} for _ in range(self.L)]

  def create_index(self):
    self.init_random_hyperplanes()
    self.W = min(self.lsh_hyperplane_ranges.values()) / float(50)

    for vector_id, vector in enumerate(self.vectors):
      for layer_id in range(self.L):
        bucket_key = self.get_bucket_key(vector, layer_id)
        if bucket_key not in self.hash_tables[layer_id]:
          self.hash_tables[layer_id][bucket_key] = set()
        self.hash_tables[layer_id][bucket_key].add(vector_id)

  def init_random_hyperplanes(self):
    # Define the origin vector with zeros, size should match the number of features (1024)
    origin_vector = np.zeros(self.vectors.shape[1])

    # Iterate through each layer and hash
    for layer_id in range(self.L):
      for hash_id in range(self.h):
        random_hyperplane_vector = []

        # Iterate through each feature
        for col in range(self.vectors.shape[1]):
          # Get min and max of the current feature across all data points
          min_val = self.vectors[:, col].min()
          max_val = self.vectors[:, col].max()

          # Get a random value between min and max
          random_val = random.uniform(min_val, max_val)
          random_hyperplane_vector.append(random_val)

        random_hyperplane_vector = np.array(random_hyperplane_vector)

        # Store the hyperplane vector and its distance from the origin
        hyperplane_key = (layer_id, hash_id)
        self.lsh_hyperplanes[hyperplane_key] = random_hyperplane_vector
        self.lsh_hyperplane_ranges[hyperplane_key] = euclidean(origin_vector, random_hyperplane_vector)

  def get_bucket_key(self, vector, layer_id):
    bucket_key = []
    for hash_id in range(self.h):
      hyperplane = self.lsh_hyperplanes[(layer_id, hash_id)]
      projected_value = self.project_on_hyperplane(vector, hyperplane)
      bucket_key.append(self.assign_vector(projected_value))
    return tuple(bucket_key)

  def project_on_hyperplane(self, input_vector, lsh_vector):
    dp = np.dot(input_vector, lsh_vector)
    if dp == 0.0:
      return 0
    projection = dp/np.dot(lsh_vector, lsh_vector)*lsh_vector
    magnitude = np.linalg.norm(projection)
    return magnitude

  def assign_vector(self, value):
    if value < 0:
      return math.floor(value/self.W)
    else:
      return math.ceil(value/self.W)

  def query(self, query_vector):
    results = []
    for layer_id in range(self.L):
      bucket_key = self.get_bucket_key(query_vector, layer_id)
      results.extend(list(self.hash_tables[layer_id].get(bucket_key, set())))
    return results

  def get_adjacent_buckets(self, bucket_key, layer_id):
    adjacent_keys = []

    mini = min(min(self.hash_tables[layer_id].keys()))
    maxi = max(max(self.hash_tables[layer_id].keys()))

    for delta in range(1, maxi - mini + 1):
      for i in range(len(bucket_key)):
        modified_key = list(bucket_key)
        modified_key[i] -= delta
      adjacent_keys.append(tuple(modified_key))
      for i in range(len(bucket_key)):
        modified_key = list(bucket_key)
        modified_key[i] += delta
      adjacent_keys.append(tuple(modified_key))

    return adjacent_keys

  def expanded_lsh_query(self, query_vector, t):
    results = self.query(query_vector)

    # Check if we need to expand the search
    if len(set(results)) >= t:
      return results

    # Expand to adjacent buckets
    for layer_id in range(self.L):
      bucket_key = self.get_bucket_key(query_vector, layer_id)
      for adj_key in self.get_adjacent_buckets(bucket_key, layer_id):
        if adj_key in self.hash_tables[layer_id].keys() and adj_key != bucket_key:
          results.extend(list(self.hash_tables[layer_id].get(adj_key, set())))
          if len(set(results)) >= t:
              return results

    return results

  def print_bucket_sizes(self):
    for layer_id, layer in enumerate(self.hash_tables):
      print(f"Layer {layer_id}:")
      for bucket_key, bucket in layer.items():
        print(f"  Bucket {bucket_key}: {len(bucket)} images")


Below is the implementation of task 4b

In [ ]:
def save_lsh_index_to_file(lsh, filename):
  with open(filename, 'wb') as file:
    pickle.dump(lsh, file)

In [ ]:
numLayers = int(input("Enter the number of Layers, L: "))
numHashesPerLayer = int(input("Enter the number of hashes per layer, h: "))

input_vectors = np.array(data["layer_3"].tolist())

lsh = LSH(numLayers, numHashesPerLayer, input_vectors)
lsh.create_index()

lsh_index_file_name = f"{path}/LSH_index_structures/lsh_index_structure_{numLayers}_{numHashesPerLayer}.pkl"
save_lsh_index_to_file(lsh, lsh_index_file_name)

Enter the number of Layers, L: 3
Enter the number of hashes per layer, h: 10


In [ ]:
lsh.print_bucket_sizes()

Layer 0:
  Bucket (45, 45, 45, 45, 45, 45, 45, 45, 45, 45): 134 images
  Bucket (46, 46, 46, 46, 46, 46, 46, 46, 46, 46): 43 images
  Bucket (44, 44, 44, 44, 44, 44, 44, 44, 44, 44): 442 images
  Bucket (44, 44, 44, 44, 43, 44, 44, 44, 44, 44): 50 images
  Bucket (44, 44, 44, 44, 44, 45, 44, 44, 44, 45): 1 images
  Bucket (43, 43, 43, 43, 43, 43, 43, 43, 43, 43): 790 images
  Bucket (44, 45, 45, 44, 44, 45, 45, 45, 45, 45): 3 images
  Bucket (43, 44, 44, 44, 43, 44, 44, 44, 44, 44): 4 images
  Bucket (43, 43, 43, 43, 43, 43, 43, 43, 44, 43): 56 images
  Bucket (43, 43, 43, 43, 43, 43, 43, 44, 43, 44): 1 images
  Bucket (44, 44, 45, 44, 44, 45, 44, 45, 45, 45): 2 images
  Bucket (44, 44, 44, 44, 44, 45, 44, 45, 45, 44): 1 images
  Bucket (44, 44, 44, 44, 44, 44, 44, 44, 45, 44): 18 images
  Bucket (45, 45, 45, 45, 44, 45, 45, 45, 45, 45): 23 images
  Bucket (44, 44, 44, 44, 44, 45, 44, 45, 45, 45): 2 images
  Bucket (42, 42, 42, 42, 42, 42, 42, 42, 42, 42): 768 images
  Bucket (43, 44, 